In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat
import pyomo.environ as pe

# Para b_media

# Intervalo entorno a la media

Importar datos del problema

In [2]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 

In [3]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_media.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_media[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_media[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b_media[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmpt4s2ix4v.pyomo.lp
Reading time = 0.02 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros
Model fingerprint: 0x3ee11f9b
Coefficient statistics:
  Matrix range     [1e-01, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 2e+03]
Presolve removed 10 rows and 5 columns
Presolve time: 0.02s
Presolved: 3 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1613118e+01   0.000000e+00   0.000000e+00      0s
       0    2.1613118e+01   0.0000

# Intervalo [min max]

In [4]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 

In [5]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_MinMax.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_MinMax[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_MinMax[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b_media[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmpsm4s6w6q.pyomo.lp
Reading time = 0.01 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros
Model fingerprint: 0x213b86e7
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 2e+03]
Presolve removed 10 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3545833e+01   0.000000e+00   0.000000e+00      0s
       0    1.3545833e+01   0.0000

# Para b_max

# Intervalo entorno a la media

Importar datos del problema

In [6]:
b_max = data['b_max'].flatten()

In [7]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_media.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_media[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_media[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b_max[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmpcux13ecy.pyomo.lp
Reading time = 0.02 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros
Model fingerprint: 0x10c7ece4
Coefficient statistics:
  Matrix range     [1e-01, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+03]
Presolve removed 10 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9145465e+01   0.000000e+00   0.000000e+00      0s
       0    2.9145465e+01   0.0000

# Intervalo [min max]

In [8]:
b_max = data['b_max'].flatten()

In [9]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_MinMax.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_MinMax[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_MinMax[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b_max[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmpxpnsb68r.pyomo.lp
Reading time = 0.01 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros
Model fingerprint: 0xf4098e4c
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+03]
Presolve removed 10 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8266667e+01   8.746652e+00   0.000000e+00      0s
       3    1.7978007e+01   0.0000